In [1]:
# importing libraries
import numpy as np
import pandas as pd
import json
import requests as rq
from datetime import datetime


In [2]:
## install necessary package
# !pip install holidays

In [3]:
import holidays as hld

In [4]:
# reading data
orders = pd.read_csv('orders.csv')
delays = pd.read_csv('partners_delays.csv')

In [5]:
# checking for orders df types
orders.dtypes

delivery_area_id     int64
date                object
orders_cnt           int64
dtype: object

In [6]:
# checking for delays df types
delays.dtypes

delivery_area_id      int64
dttm                 object
partners_cnt        float64
delay_rate          float64
dtype: object

In [7]:
# converting string dates to datetime
orders['date'] = pd.to_datetime(orders['date'])
delays['dttm'] = pd.to_datetime(delays['dttm'])

In [8]:
# deriving datetime columns
orders_dates = orders['date']
delays_dates = delays['dttm']

In [9]:
# # deriving month from datetime col
# orders['month'] = orders_dates.apply(lambda x: x.month)
# delays['month'] = delays_dates.apply(lambda x: x.month)

In [10]:
# deriving day from datetime col
orders['day'] = orders_dates.apply(lambda x: x.day)
delays['day'] = delays_dates.apply(lambda x: x.day)

In [11]:
# deriving hour from datetime col
orders['hour'] = orders_dates.apply(lambda x: x.hour)
delays['hour'] = delays_dates.apply(lambda x: x.hour)

In [12]:
# deriving weekday from datetime col
orders['weekday'] = orders_dates.apply(lambda x: x.weekday())
delays['weekday'] = delays_dates.apply(lambda x: x.weekday())

In [13]:
# deriving day of year from datetime col
orders['day_of_year'] = orders_dates.apply(lambda x: x.dayofyear)
delays['day_of_year'] = delays_dates.apply(lambda x: x.dayofyear)

In [14]:
#derive frequency of particular day in data

day_freq_orders = orders['day'].value_counts(normalize=True).to_dict()
orders['day_freq'] = orders['day'].map(day_freq_orders)

day_freq_delays = delays['day'].value_counts(normalize=True).to_dict()
delays['day_freq'] = delays['day'].map(day_freq_delays)

In [15]:
# #derive frequency of particular month in data

# month_freq_orders = orders['month'].value_counts(normalize=True).to_dict()
# orders['month_freq'] = orders['month'].map(month_freq_orders)

# month_freq_delays = delays['month'].value_counts(normalize=True).to_dict()
# delays['month_freq'] = delays['month'].map(month_freq_delays)

In [16]:
#derive frequency of particular weekday in data

weekday_freq_orders = orders['weekday'].value_counts(normalize=True).to_dict()
orders['weekday_freq'] = orders['weekday'].map(weekday_freq_orders)

weekday_freq_delays = delays['weekday'].value_counts(normalize=True).to_dict()
delays['weekday_freq'] = delays['weekday'].map(weekday_freq_delays)

In [17]:
# deriving datetime columns
orders_dates = orders['date']
delays_dates = delays['dttm']

In [18]:
# deriving russian official holidays
holidays = [date[0] for date in hld.Russia(years=2021).items()]

In [19]:
#counting days until and before some official holiday day for orders

orders_dates = orders_dates.to_frame().sort_values(by='date')

df_holidays = pd.DataFrame({'holidays': holidays})
df_holidays['holidays'] = pd.to_datetime(df_holidays['holidays'])

orders_dates = pd.merge_asof(orders_dates, df_holidays, left_on='date', right_on='holidays', direction='forward')
orders_dates = pd.merge_asof(orders_dates, df_holidays, left_on='date', right_on='holidays')

orders_dates['days_until_holiday'] = orders_dates.pop('holidays_x').sub(orders_dates['date']).dt.days
orders_dates['days_since_holiday'] = orders_dates['date'].sub(orders_dates.pop('holidays_y')).dt.days

orders_dates = orders_dates.drop_duplicates()

In [20]:
#counting days until and before some official holiday day for delays

delays_dates = delays_dates.to_frame().sort_values(by='dttm')

df_holidays = pd.DataFrame({'holidays': holidays})
df_holidays['holidays'] = pd.to_datetime(df_holidays['holidays'])

delays_dates = pd.merge_asof(delays_dates, df_holidays, left_on='dttm', right_on='holidays', direction='forward')
delays_dates = pd.merge_asof(delays_dates, df_holidays, left_on='dttm', right_on='holidays')

delays_dates['days_until_holiday'] = delays_dates.pop('holidays_x').sub(delays_dates['dttm']).dt.days
delays_dates['days_since_holiday'] = delays_dates['dttm'].sub(delays_dates.pop('holidays_y')).dt.days

delays_dates =  delays_dates.drop_duplicates()

In [21]:
# merging 
orders = pd.merge(orders, orders_dates, how='left', on='date')
delays = pd.merge(delays, delays_dates, how='left', on='dttm')

---------------

In [22]:
# deriving datetime columns
orders_dates = orders['date']
delays_dates = delays['dttm']

In [23]:
# getting the data for any non-working day (includes holidays and weekends)

url = 'https://raw.githubusercontent.com/d10xa/holidays-calendar/master/json/consultant2021.json'
response = rq.get(url)
non_working = json.loads(response.text)

In [24]:
# converting the non_working data
for days in non_working:
    non_working[days] = pd.to_datetime(non_working.get(days)).date

In [25]:
# if the day is holiday
orders['is_holiday'] = orders_dates.apply(lambda x: x.date() in non_working['holidays']) * 1
delays['is_holiday'] = delays_dates.apply(lambda x: x.date() in non_working['holidays']) * 1

In [26]:
# if the day is weekend
orders['is_weekend'] = ((orders['weekday']==5) | (orders['weekday']==6)) * 1
delays['is_weekend'] = ((delays['weekday']==5) | (delays['weekday']==6)) * 1

In [27]:
#counting days until and before some official non-working day for orders

orders_dates = orders_dates.to_frame().sort_values(by='date')

sorted_nonworking = np.sort(np.concatenate((non_working['holidays'], non_working['nowork'])))

df_non_working = pd.DataFrame({'non_working': sorted_nonworking})
df_non_working['non_working'] = pd.to_datetime(df_non_working['non_working'])

orders_dates = pd.merge_asof(orders_dates, df_non_working, left_on='date', right_on='non_working', direction='forward')
orders_dates = pd.merge_asof(orders_dates, df_non_working, left_on='date', right_on='non_working')

orders_dates['days_until_nonworking'] = orders_dates.pop('non_working_x').sub(orders_dates['date']).dt.days
orders_dates['days_since_nonworking'] = orders_dates['date'].sub(orders_dates.pop('non_working_y')).dt.days

orders_dates = orders_dates.drop_duplicates()

In [28]:
#counting days until and before some official non-working day for delays

delays_dates = delays_dates.to_frame().sort_values(by='dttm')

sorted_nonworking = np.sort(np.concatenate((non_working['holidays'], non_working['nowork'])))

df_non_working = pd.DataFrame({'non_working': sorted_nonworking})
df_non_working['non_working'] = pd.to_datetime(df_non_working['non_working'])

delays_dates = pd.merge_asof(delays_dates, df_non_working, left_on='dttm', right_on='non_working', direction='forward')
delays_dates = pd.merge_asof(delays_dates, df_non_working, left_on='dttm', right_on='non_working')

delays_dates['days_until_nonworking'] = delays_dates.pop('non_working_x').sub(delays_dates['dttm']).dt.days
delays_dates['days_since_nonworking'] = delays_dates['dttm'].sub(delays_dates.pop('non_working_y')).dt.days

delays_dates =  delays_dates.drop_duplicates()

In [29]:
# merging 

orders = pd.merge(orders, orders_dates, how='left', on='date')
delays = pd.merge(delays, delays_dates, how='left', on='dttm')

In [30]:
#dropping unnecessary cols

orders = orders.drop(columns='date')
delays = delays.drop(columns='dttm')

In [31]:
orders.sample(5)

,delivery_area_id,orders_cnt,day,hour,weekday,day_of_year,day_freq,weekday_freq,days_until_holiday,days_since_holiday,is_holiday,is_weekend,days_until_nonworking,days_since_nonworking
548203,331,2,25,12,5,268,0.034719,0.144994,39,105,1,1,0,0
237116,97,1,6,10,2,279,0.030693,0.139205,28,116,0,0,2,3
782062,499,4,22,12,4,295,0.033911,0.141950,12,132,0,0,0,5
181311,66,7,18,13,0,291,0.033291,0.141917,16,128,0,0,4,1
547826,331,3,15,19,6,227,0.032977,0.147145,80,64,1,1,5,0


In [32]:
delays.sample(5)

,delivery_area_id,partners_cnt,delay_rate,day,hour,weekday,day_of_year,day_freq,weekday_freq,days_until_holiday,days_since_holiday,is_holiday,is_weekend,days_until_nonworking,days_since_nonworking
844405,376,1.0,0.0,29,13,6,241,0.035774,0.143506,66,78,1,1,5,0
742207,325,1.0,0.0,29,21,1,180,0.035774,0.144958,127,17,0,0,3,2
107338,31,1.0,0.0,14,21,5,226,0.031761,0.143143,81,63,1,1,0,0
86362,25,3.0,0.0,30,20,6,150,0.036027,0.143506,12,21,1,1,5,0
811486,360,2.0,0.0,19,18,3,231,0.032834,0.141769,76,68,0,0,1,4


In [33]:
orders.to_csv('orders_extracted.csv')
delays.to_csv('delays_extracted.csv')